In [ ]:
import sys
sys.path.append('../util')

from util import get_max_db_id
from util import get_time_span
from util import collect_segments

In [ ]:
collect_segments('STT_TT', 'after_split', 'segments_tt')

In [ ]:
! aws s3 cp segments_tt_ga  s3://monlam.ai.stt/stt_pecha_tools/ --recursive

In [ ]:
STT_TT_GA = 11
STT_TT_GB = 12

group_id = STT_TT_GA
last_db_id = get_max_db_id()

target_directory = f"segments_tt"

In [ ]:
from transformers import pipeline
from tqdm.auto import tqdm

from pathlib import Path

target_path = Path(target_directory)

rows = []
generator = pipeline(model="openpecha/wav2vec2_run8", device=0)

for file in tqdm(target_path.glob('*.wav'), total=len(list(target_path.glob('*.wav')))):
    inf = generator(str(file))["text"]
    rows.append([file.stem, f"https://d38pmlk0v88drf.cloudfront.net/stt_pecha_tools/{file.name}", inf, get_time_span(str(file.name))])

In [ ]:
import pandas as pd
df = pd.DataFrame(rows, columns =['file_name', 'url', 'inference_transcript', 'audio_duration'])

In [ ]:
df[['inference_transcript','url']].iloc[0:10].to_dict()

In [ ]:
df['group_id'] = group_id
df['state'] = 'imported'
df['id'] = df.index + last_db_id + 1

In [ ]:
df.head()

In [ ]:
df.to_csv(f"segments_tt_{group_id}.csv", index=False)